# Flash Loans

# Objective

- Perform a profitable arbitrage between a Uniswap and Sushiswap pool using a Flash Loan from Aave

# Background

An [arbitrage](https://www.investopedia.com/terms/a/arbitrage.asp) involves taking advantage of different prices for the same asset on different exchanges. For example, if some asset was trading for \\$1 on exchange A and trading for \\$2 on exchange B, you could buy it on exchange A for \\$1 and then sell it on exchange B for \\$2, thus netting you a profit from the difference. Due to [slippage](https://www.investopedia.com/terms/s/slippage.asp), performing the arbitrage will eventually cause the prices on the two exchanges will reach an equilibrium at which point no profitable arbitrage opportunity remains. Note that these are general finance concepts and are not specific to DeFi.

Since DeFi exchanges are often in the form of permissionless smart contracts, anyone can perform an arbitrage by buying/selling through calling the contracts. We can also guarantee through code that the trades that make up our arbitrage result in a profit, otherwise we can revert the entire transaction (ie. if the prices have changed in the meantime).

Notice that performing an arbitrage requires initial capital as you need to buy the asset on the venue where it's trading cheap before you can sell it on the other exchange. This is where Flash Loans come in. Flash Loans allow you to borrow an essentially unlimited amount of an asset using zero collateral, as long as you return the full amount borrowed *within the same transaction*. This is unique to DeFi and has no comparable analogy in traditional finance.

Flash Loans typically operate similar to the following:

- Your contract calls the public Flash Loan function (eg. `flashLoan`) on the protocol's contract
- The protocol's contract sends you the requested funds and then calls a designated callback function defined in your contract (eg. `executeOperation`)
- The protocol's contract ensures that you have enough funds to repay the borrowed amount (plus a fee) and reclaims those funds

The `executeOperation` function in your contract can perform any arbitrary logic using the funds. Once it finishes, execution returns back to the calling protocol contract where it can transfer the borrowed funds back to itself and ensure the entire amount is repaid. If you no longer have enough funds to repay the borrowed amount by the last step, the entire transaction will fail. As a result, the protocol can let anyone borrow any amount with no risk of losing assets.

Flash Loan providers are typically protocols with large amounts of deposits, since that's the capital being used for the loans. One of the most popular examples is the lending protocol [Aave](https://docs.aave.com/developers/guides/flash-loans).

# Overview

In this lab, we are going to perform a profitable arbitrage between the Uniswap and SushiSwap `wBTC/LUSD` pools. We have created a `wBTC/LUSD` pool for both protocols on the Goerli Testnet and have artificially created an arbitrage opportunity by manipulating the swap rates in each pool to be different.

We are going to Flash Loan `wBTC` from Aave in order to perform the arbitrage between two pools without needing to have any of either token to start with.

The arbitrage is as follows (assume `wBTC/LUSD` is cheaper on Uniswap):

- Flash Loan X amount of `wBTC` from Aave (we now have X `wBTC` and 0 `LUSD`)
- Swap all our `wBTC` for `LUSD` on the SushiSwap pool contract (we now have 0 `wBTC` and Y `LUSD`)
- Swap all our `LUSD` for `wBTC` on the Uniswap pool contract (we now have Z `wBTC` and 0 `LUSD`)
- Repay the borrowed X `wBTC` + a small fee to the Aave contract (we now have Z-X-fee `wBTC` and 0 `LUSD`)

If we cannot afford to repay what we borrowed in the last step, the whole transaction will revert. If the transaction does not revert, then we must have made a profit! Note that our profit is denominated in `wBTC`. Optionally, we could include a final step to swap our profit into some other token, such as a stablecoin.

Please read the [Aave docs](https://docs.aave.com/developers/guides/flash-loans) on Flash Loans to understand how they work in practice. Importantly, note that:
- We want to use the [`flashLoanSimple`](https://docs.aave.com/developers/core-contracts/pool#flashloansimple) function defined in the Aave Pool contract
- The `flashLoanSimple` will call the callback function in our contract named `executeOperation`
- We do not need to transfer the borrowed funds back; the Aave contract will automatically take back the borrowed funds (assuming we have enough funds)

In [ ]:
### SETUP ###

from web3 import Web3
import time
import json

In [ ]:
### Constants ###

RPC_URL = "YOUR GOERLI RPC URL HERE"

# These are the wBTC/LUSD pools we created
UNISWAP_POOL_ADDRESS = "0x4127590e224411d002FFc8ead0c3D94C6B6b0a4f"
SUSHISWAP_POOL_ADDRESS = "0xA5342DAaF01013b167d4e42188E96E1b8eF61DA4"

WBTC_ADDRESS = "0x45AC379F019E48ca5dAC02E54F406F99F5088099"
LUSD_ADDRESS = "0x4966Bb6Cd9f3e042331b0798525b7970eFB0D94A"

In [ ]:
# Replace with your private key
PRIVATE_KEY = "YOUR PRIVATE KEY HERE"

In [ ]:
# ABIs

ERC20_ABI = '[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transfer","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":true,"name":"owner","type":"address"},{"indexed":true,"name":"spender","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Transfer","type":"event"}]'
UNISWAP_V2_PAIR_ABI = '[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount0Out","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1Out","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Swap","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint112","name":"reserve0","type":"uint112"},{"indexed":false,"internalType":"uint112","name":"reserve1","type":"uint112"}],"name":"Sync","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"constant":true,"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"MINIMUM_LIQUIDITY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"burn","outputs":[{"internalType":"uint256","name":"amount0","type":"uint256"},{"internalType":"uint256","name":"amount1","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getReserves","outputs":[{"internalType":"uint112","name":"reserve0","type":"uint112"},{"internalType":"uint112","name":"reserve1","type":"uint112"},{"internalType":"uint32","name":"blockTimestampLast","type":"uint32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"initialize","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"kLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"mint","outputs":[{"internalType":"uint256","name":"liquidity","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"price0CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"price1CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"skim","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount0Out","type":"uint256"},{"internalType":"uint256","name":"amount1Out","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"swap","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[],"name":"sync","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"}]'
FLASH_LOAN_ARBITRAGE_ABI = '[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"arbitrage","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"premium","type":"uint256"},{"internalType":"address","name":"initiator","type":"address"},{"internalType":"bytes","name":"params","type":"bytes"}],"name":"executeOperation","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"}]'

In [ ]:
w3 = Web3(Web3.HTTPProvider(RPC_URL))

# Setup contracts
wbtc_contract = w3.eth.contract(address=WBTC_ADDRESS, abi=ERC20_ABI)
lusd_contract = w3.eth.contract(address=LUSD_ADDRESS, abi=ERC20_ABI)
uniswap_pool_contract = w3.eth.contract(address=UNISWAP_POOL_ADDRESS, abi=UNISWAP_V2_PAIR_ABI)
sushiswap_pool_contract = w3.eth.contract(address=SUSHISWAP_POOL_ADDRESS, abi=UNISWAP_V2_PAIR_ABI)

# Fetch token decimals
wbtc_decimals = wbtc_contract.functions.decimals().call()
lusd_decimals = lusd_contract.functions.decimals().call()

# Returns the live quotes from the Uniswap and SushiSwap pool contracts
def get_current_quotes():
    uniswap_wbtc_amount, uniswap_lusd_amount, _ = uniswap_pool_contract.functions.getReserves().call()
    sushiswap_wbtc_amount, sushiswap_lusd_amount, _ = sushiswap_pool_contract.functions.getReserves().call()

    uniswap_rate = uniswap_lusd_amount / uniswap_wbtc_amount * 10**wbtc_decimals / 10**lusd_decimals
    sushiswap_rate = sushiswap_lusd_amount / sushiswap_wbtc_amount * 10**wbtc_decimals / 10**lusd_decimals
    
    return uniswap_rate, sushiswap_rate

Let's get the live swap rates for both the Uniswap and Sushiswap `wBTC/LUSD` pools to see if there is an arbitrage opportunity:

In [ ]:
uniswap_rate, sushiswap_rate = get_current_quotes()
print(f"wBTC/LUSD exchange rate for Uniswap pool: {uniswap_rate}")
print(f"wBTC/LUSD exchange rate for SushiSwap pool: {sushiswap_rate}")

As we see here, `wBTC` is worth a lot more `LUSD` on SushiSwap than it is on Uniswap right now.

To take advantage of this, we want to *borrow* `wBTC`, *sell* `wBTC` in exchange for `LUSD` on SushiSwap, *buy* `wBTC` using the `LUSD` on Uniswap, return the original amount of `wBTC` we borrowed, and then our profit is the remaining `wBTC`.

Furthermore, we want to use a Flash Loan for that first step, so we don't have to provide any of our own capital. To use a Flash Loan, we must call `flashLoanSimple` on the Aave contract, and that call must come from another contract since we must have a callback function named `executeOperation` for it to call, so our arbitrage logic must take place entirely within a smart contract rather than using an EOA and multiple transactions using `web3.py`.

## Contract

The below contract implements all the logic described up to this point. The contract is deployed on Goerli at `0xa70BbA21c16c21CF129E3371F37cb0ec62fE4830` and [verified on Etherscan](https://goerli.etherscan.io/address/0xa70bba21c16c21cf129e3371f37cb0ec62fe4830#code), so you can view its source code there as well. The `arbitrage` function takes no arguments and is what triggers all the execution.

Please read through the smart contract and understand each of the steps - this is where all the implementation is!

```javascript
// SPDX-License-Identifier: UNLICENSED
pragma solidity ^0.8.17;

contract FlashLoanArbitrage {
    /*
        This contract is very simplistic and for demonstration purposes only. Here, we always sell on the SushiSwap
        pool and buy back on the Uniswap pool (ie. we assume the Uniswap pool is always the one trading cheaper). In a
        real contract, some things you might want to do include not hard coding the tokens or pools, dynamically
        finding the optimal arbitrage path and amounts, and checking to ensure the arbitrage is profitable, just a few
        examples.
    */
    uint256 constant MAX_UINT = 2**256 - 1;

    // Address of the wBTC token contract
    address constant WBTC_TOKEN = 0x45AC379F019E48ca5dAC02E54F406F99F5088099;

    // Address of the LUSD token contract
    address constant LUSD_TOKEN = 0x4966Bb6Cd9f3e042331b0798525b7970eFB0D94A;

    // Addresses of the SushiSwap and Uniswap V2 routers we will be interacting with to perform the swaps
    address constant SUSHISWAP_ROUTER = 0x1b02dA8Cb0d097eB8D57A175b88c7D8b47997506;
    address constant UNISWAP_ROUTER = 0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D;

    // Address of the Aave pool we will be flash loaning from
    address constant AAVE_POOL = 0x7b5C526B7F8dfdff278b4a3e045083FBA4028790;

    constructor() {
        // Approve the SushiSwap router for all our wBTC
        IERC20(WBTC_TOKEN).approve(SUSHISWAP_ROUTER, MAX_UINT);

        // Approve the Uniswap V2 router for all our LUSD
        IERC20(LUSD_TOKEN).approve(UNISWAP_ROUTER, MAX_UINT);

        // Approve the Aave pool for all our wBTC (needed to return the flash loan)
        IERC20(WBTC_TOKEN).approve(AAVE_POOL, MAX_UINT);
    }

    /**
     * @dev The entry point to this contract to perform the arbitrage. In this simplistic example,
            we flash loan a constant amount of wBTC from the Aave pool and always perform swap it
            for LUSD using the SushiSwap pool, and then swap the LUSD back to wBTC using the
            Uniswap pool.
     */
    function arbitrage() external {
        // Call the flash loan function in the Aave pool contract
        // See Aave docs for more information:
        // https://docs.aave.com/developers/core-contracts/pool#flashloansimple
        IPool(AAVE_POOL).flashLoanSimple(
            address(this), // receiverAddress: this contract
            WBTC_TOKEN, // asset: wBTC
            1000000, // amount: 0.01 wBTC
            "", // params: not relevant here
            0 // referralCode: not relevant here
        );

        // Any amount of wBTC in the contract is our profit
        IERC20 wbtc = IERC20(WBTC_TOKEN);
        uint256 profit = IERC20(WBTC_TOKEN).balanceOf(address(this));

        // Send profit to caller
        wbtc.transfer(msg.sender, profit);
    }

    /**
     * @dev The callback function that will be invoked by the Aave contract.
     * @param asset The address of the flash-borrowed asset
     * @param amount The amount of the flash-borrowed asset
     * @param premium The fee of the flash-borrowed asset
     * @param initiator The address of the flashloan initiator
     * @param params The byte-encoded params passed when initiating the flashloan
     * @return True if the execution of the operation succeeds, false otherwise
     */
    function executeOperation(
        address asset,
        uint256 amount,
        uint256 premium,
        address initiator,
        bytes calldata params
    ) external returns (bool) {
        // We interact with the routers to perform the swap, rather than the actual pool contracts directly
        IUniswapV2Router02 sushiswapRouter = IUniswapV2Router02(SUSHISWAP_ROUTER);
        IUniswapV2Router02 uniswapRouter = IUniswapV2Router02(UNISWAP_ROUTER);


        /***** Swap all our wBTC for LUSD using the SushiSwap pool *****/

        // Construct swap path
        address[] memory path = new address[](2);
        path[0] = WBTC_TOKEN;
        path[1] = LUSD_TOKEN;

        // Swap the 0.01 wBTC we flash loaned into LUSD
        sushiswapRouter.swapExactTokensForTokens(
            amount, // amountIn: amount of wBTC
            0, // amountOutMin: minimum acceptable LUSD received
            path, // path: swap path
            address(this), // to: this contract
            block.timestamp // deadline: latest acceptable time to complete the swap by
        );


        /***** Swap all our LUSD for wBTC using the Uniswap pool *****/

        // Construct swap path
        path[0] = LUSD_TOKEN;
        path[1] = WBTC_TOKEN;

        // Swap all the LUSD we received from the previous swap back into wBTC
        uniswapRouter.swapExactTokensForTokens(
            IERC20(LUSD_TOKEN).balanceOf(address(this)), // amountIn: amount of LUSD
            0, // amountOutMin: minimum acceptable wBTC received
            path, // path: swap path
            address(this), // to: this contract
            block.timestamp // deadline: latest acceptable time to complete the swap by
        );

        return true;
    }
}

/**
 * @dev Partial interface for a Uniswap V2 router contract (SushiSwap pools use the same interface).
 */
interface IUniswapV2Router02 {
    function swapExactTokensForTokens(
        uint256 amountIn,
        uint256 amountOutMin,
        address[] calldata path,
        address to,
        uint256 deadline
    ) external returns (uint256[] memory amounts);
}

/**
 * @dev Partial interface for an Aave V3 pool contract.
 */
interface IPool {
    function flashLoanSimple(
        address receiverAddress,
        address asset,
        uint256 amount,
        bytes calldata params,
        uint16 referralCode
    ) external;
}

/**
 * @dev Partial interface for an ERC-20 token.
 */
interface IERC20 {
    function balanceOf(address account) external view returns (uint256);

    function transfer(address to, uint256 amount) external returns (bool);

    function approve(address spender, uint256 amount) external returns (bool);
}
```

## Trying it live
Now that the contract is deployed, we can call the `arbitrage` function from an EOA to execute the logic. Note that you don't need any `wBTC` or `LUSD` in your wallet to do this - that's the point of the Flash Loan!

In [ ]:
FLASH_LOAN_ARBITRAGE_ADDRESS = "0xa70BbA21c16c21CF129E3371F37cb0ec62fE4830"
flash_loan_arbitrage_contract = w3.eth.contract(address=FLASH_LOAN_ARBITRAGE_ADDRESS, abi=FLASH_LOAN_ARBITRAGE_ABI)
public_address = w3.eth.account.from_key(PRIVATE_KEY).address

In [ ]:
### THIS CELL MAKES A TRANSACTION ###

# Query wBTC balance and swap rates before doing the arbitrage
wbtc_balance_before = wbtc_contract.functions.balanceOf(public_address).call()
uniswap_rate_before, sushiswap_rate_before = get_current_quotes()

# Doing the arbitrage - all we do is call the `arbitrage` function in the contract
txn = flash_loan_arbitrage_contract.functions.arbitrage().build_transaction({'nonce':w3.eth.getTransactionCount(public_address),'from': public_address})
txn_signed = w3.eth.account.signTransaction(txn, PRIVATE_KEY)
txn_hash = w3.eth.send_raw_transaction(txn_signed.rawTransaction)
w3.eth.waitForTransactionReceipt(txn_hash)
time.sleep(10) # to ensure the below queries don't return stale values

# Query wBTC balance and swap rates after doing the arbitrage
wbtc_balance_after = wbtc_contract.functions.balanceOf(public_address).call()
uniswap_rate_after, sushiswap_rate_after = get_current_quotes()

In [ ]:
print(f"Transaction hash:\t {w3.toHex(txn_hash)}")
print(f"Etherscan link:\t\t https://goerli.etherscan.io/tx/{w3.toHex(txn_hash)}\n")
print(f"Your wBTC balance before:\t{wbtc_balance_before / 10**wbtc_decimals}")
print(f"Your wBTC balance after:\t{wbtc_balance_after / 10**wbtc_decimals}")
print(f"Your wBTC profit:\t\t{(wbtc_balance_after - wbtc_balance_before) / 10**wbtc_decimals}\n")
print(f"wBTC/LUSD exchange rate for Uniswap pool before:\t{uniswap_rate_before}")
print(f"wBTC/LUSD exchange rate for SushiSwap pool before:\t{sushiswap_rate_before}\n")
print(f"wBTC/LUSD exchange rate for Uniswap pool after:\t\t{uniswap_rate_after}")
print(f"wBTC/LUSD exchange rate for SushiSwap pool after:\t{sushiswap_rate_after}\n")

If all went well, you should see that you made a small `wBTC` profit, all without needing any of your own capital! Also, note that the exchange rates between the two pools have slightly converged - this is due to slippage. After enough arbitrages (or a single appropriately-sized arbitrage, the pools will converge to an equilibrium price at which point there will be no arbitrage opportunity left.) For the purposes of this lab, please refrain from running the above cell too many times. We have purposely made the swap amount small so we can maintain the difference in rates and keep the arbitrage opportunity for other students to run the lab.

## A note on MEV bots

This example illustrates a classic type of MEV. If you were trying to arbitrage for profit, you would likely want to calculate the optimal amount to swap in order to bring the swap rates to the same in each pool, thus giving you the maximum profit possible and eliminating the arbitrage opportunity. Think about how you could do this off-chain (hint: if you know the reserve values of each assets in the pools, you could use the x * y = k formula to compute the resulting amount in each pool after swapping). You also want to be the first person to perform the arbitrage as soon as an opportunity forms, as the profiting from the arbitrage closes the opportunity. In practice, this type of simple arbitrage is extremely competitive and result in very high validator bribes through Flashbots bribes.